In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/Users/siyuyang/Source/repos/GitHub_MSFT/CameraTraps')  # append this repo to PYTHONPATH

In [ ]:
import json
import os
from collections import Counter, defaultdict
from random import sample
import math

from tqdm import tqdm
from unidecode import unidecode 

from data_management.megadb.schema import sequences_schema_check
from data_management.annotations.add_bounding_boxes_to_megadb import *
from data_management.megadb.converters.cct_to_megadb import make_cct_embedded, process_sequences, write_json

# Example importer

This notebook is a template for how new datasets can be formatted for ingestion into the database.

The ideal dataset has both **location** and **sequence** information, in addition to any species or bounding box labels.

Give the path to a JSON file where output from this script will be written to. You can then take this file to the .Net app for ingestion to the database.

In [ ]:
path_to_output = '/path/to/sequence/items.json'  

**Name of the dataset**

All lower-case; use `_` not `-` if needed

A `dataset` is a set of image sequences that are located in the same storage container at the same path (there can be sibling folders at that path which are not a part of this dataset). Since blob storage access is mostly container-based, if the access level on this dataset is `internal`, then it should be in a container all by itself or be in a container where all folders are internal datasets. Thus the subset of human or sensitive images from a larger dataset is usually a separate `dataset`, usually named `largedataset_private`.

In [ ]:
dataset_name = 'new_dataset_name'

## Step 0 - Add an entry to the `datasets` table

For format, see https://github.com/microsoft/CameraTraps/tree/master/data_management/megadb section on `datasets` table.

To insert a new entry to the `datasets` table, use `data_management/megadb/query_and_upsert_examples/upsert_items.ipynb` or do that in Azure Portal (not currently working).

## Step 1 - Prepare the `sequence` objects to insert into the database

Two possibilities...

### Step 1a - If you have metadata in COCO Camera Traps (CCT) format already...

For a dataset, you probably have one or two JSONs in the CCT format, one containing image-level species labels and another containing bounding box annotations. Here we combine them and embed any annotation items into the image items.

In [ ]:
# path to the CCT json, or a loaded json object
path_to_image_cct = 'path/to/cct.json'  # set to None if not available
path_to_bbox_cct = 'path/to/cct_bboxes.json'  # set to None if not available
assert not (path_to_image_cct is None and path_to_bbox_cct is None)

You might like to process the resulting `embedded` dataset a little more:

- For image entries that do not have species label but have bounding box annotations, you can add a `species` field to the `annotations` field of each item in the list `embedded`, according to the `category` field of the first `bbox` item in `annotations`:
    - If `e['annotations']['bbox'][category']` is `person`, assign `['human']` to `e['annotations']['species']`. Note that it needs to be a list (of one item).
    - If `animal`, assign `['unidentified']`

In [ ]:
%%time

embedded = make_cct_embedded(image_db=path_to_image_cct, bbox_db=path_to_bbox_cct)

In the following step, properties will be moved to the highest level that is still correct, i.e. if a property at the image-level always has the smae value for all images in a sequence, it will be moved to be a sequence-level property.

If a sequence-level property has the same value throughout this dataset (often 'rights holder'), it will be removed from the `sequence` objects. A message about this will be printed, and you should add that property and its (constant) value to this dataset's entry in the `datasets` table.

In [ ]:
%%time

sequences = process_sequences(embedded, dataset_name)

In [ ]:
# sample some sequences to make sure they are what you expect
sample(sequences, 10)

### Step 1b - If you're starting from scratch...

i.e. starting from metadata in another format, e.g. spreadsheets, CSVs, 

you would need to compile the metadata into a list of `sequence` items - format specified at https://github.com/microsoft/CameraTraps/tree/master/data_management/megadb.

Call the result `sequences`

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [ ]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

## Step 3 - Add any iMerit bbox annotations

In [ ]:
annotation_path = 'path/to/folder_containing_imerit_csvs'  # dataset_name will be the part before `+`

In [ ]:
image_filename_to_bboxes = extract_annotations(annotation_path, dataset_name)

In [ ]:
sequences, images_updated = add_annotations_to_sequences(sequences, image_filename_to_bboxes)

In [ ]:
# sample some sequences to make sure things look good

sample(sequences, 10)

## Step 4 - Save the `sequence` items to a file

You can now take the resulting JSON file to the .Net application for bulk insertion to the database:

In [ ]:
with open(path_to_output, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, ensure_ascii=False)

You can check that the bounding box annotations and paths to images all survived by running the `visualization/visualize_megadb.py` using the above exported file.